# Dense High-Quality 3D Reconstruction
## Vis_MVSNet Approach

Following the pipeline below, this tutorial demonstrates the process and steps we took to integrate Vis_MVSNet into the pipeline.

![pipeline](https://github.com/luisIvey05/Dense3D/blob/main/images/pipeline.png?raw=true)



The first thing to do is download the BU2 dataset located <a href="https://drive.google.com/file/d/1L7gdMz5H8jBdSpslS-U0i7fA--7UJCKA/view?usp=sharing"> here </a>.


In [ ]:
! unzip -DD -q  BU2.zip -d  /content/

The next steps are used to download COLMAP. However, due to how long it takes to calculate camera parameters and camera pose estimations it is strongly recommend to install COLMAP locally. 

In [ ]:
!sudo apt-get install git cmake build-essential libboost-program-options-dev libboost-filesystem-dev libboost-graph-dev libboost-regex-dev libboost-system-dev libboost-test-dev libeigen3-dev libsuitesparse-dev libfreeimage-dev libgoogle-glog-dev libgflags-dev libglew-dev qtbase5-dev libqt5opengl5-dev libcgal-dev libcgal-qt5-dev
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout $(git describe --tags) # Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!sudo make install
!git clone https://github.com/colmap/colmap
%cd colmap
!git checkout dev
%mkdir build
%cd build
!cmake ..
!make
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..

We then used COLMAP to calculate camera parameters and camera pose estimations on BU2 dataset.

In [ ]:
%cd /content/

!colmap feature_extractor \
    --database_path /content/BU2/database.db \
    --image_path /content/BU2/images_col

!colmap exhaustive_matcher \
    --database_path /content/BU2/database.db

!colmap mapper \
    --database_path /content/BU2/database.db \
    --image_path /content/BU2/images_col \
    --output_path /content/BU2/sparse_col

!colmap model_converter \
    --input_path /content/BU2/sparse_col/0 \
    --output_path /content/BU2/sparse_col \
    --output_type TXT

Next, we use Vis_MVSNet's colmap2mvsnet.py to convert the output of COLMAP to a format that Vis_MVSNet can read.

In [ ]:
!git clone https://github.com/jzhangbs/Vis-MVSNet.git

In [ ]:
!pip install torch==1.4.0 numpy==1.18.1 opencv-python==4.1.2.30 tqdm==4.41.1 matplotlib==3.1.3 open3d==0.9.0.0 apex==0.1

In [ ]:
%cd Vis-MVSNet
!python colmap2mvsnet.py --dense_folder /content/BU2 --max_d 256 --convert_format

For the next step, download our fine-tuned model <a href="https://drive.google.com/file/d/16vPf16wTOAV_0Z-LxV3Yc_CFi3BtSjR9/view?usp=sharing"> here </a>. We are then going to produce the predicted depth maps.

In [ ]:
!python test.py --data_root /content/BU2 --dataset_name general --num_src 4 --max_d 256 --resize 1920,1088 --crop 1920,1088 --load_path /content/mid_air --write_result --result_dir /content/BU2/output

We then fuse the probabily map and depth map to get the final depth map estimation. 

In [ ]:
!python fusion.py --data /content/output --pair /content/BU2/pair.txt --vthresh 4 --pthresh .8,.7,.8

The next step is to go into the "/content/BU2/output" directory to remove unneccsary files. 

---



In [ ]:
%cd /content/BU2/output
!rm -rf /*.txt
!rm -rf /*prob.pfm
!rm -rf /*.jpg

We then convert the pfm files to .png while scaling each pixel by 1000.

In [ ]:
import cv2
import imageio
import glob
import argparse
import shutil
import os
import numpy as np


pfm_path = "./"
png_path = "../image/"
print("[INFO] PFM_PATH {}    PNG_PATH {}   ".format(pfm_path, png_path))
pfm_files = glob.glob(pfm_path + '/*.pfm')
if os.path.exists(png_path):
  shutil.rmtree(png_path)
os.makedirs(png_path)
total = len(pfm_files)
for idx, fpath in enumerate(pfm_files):
  fname = os.path.basename(fpath)
  new_fname = os.path.splitext(fname)[0]
  new_fname += '.png'
  print("[INFO] DEPTH {} to {} ---- {} / {}".format(fname, new_fname, idx + 1, total))
  pfm = cv2.imread(fpath, cv2.IMREAD_UNCHANGED)
  print(pfm.dtype)
  pfm *= 1000
  imageio.imwrite(png_path + new_fname, pfm.astype(np.uint16))

The images and the depth directory are not temporally coherent because Vis_MVSNet establishes sequences of images to eliminate occuluded pixels. However, the new arrangement of images and depth maps cannot be fed into Open3D's reconstruction system without first removing images that are out of place in time. The final B2 images and predicted depth maps can be downloaded <a href="https://drive.google.com/file/d/1bh2WtAF6mhRhJIA1Gp68pmicakCqar4_/view?usp=share_link"> here </a>.

In [ ]:
%cd /content/
!git clone https://github.com/isl-org/Open3D.git
%cd /content/Open3D/examples/python/reconstruction_system/
!mkdir ./datasets
!mkdir ./datasets/016
!mkdir ./optional
!pip install open3d

Please, place the images and predicted depth maps directory inside "/content/Open3D/examples/python/reconstruction_system/datasets/016/" and name the folders as image and depth, respectively. Once again the final B2 image and predicted depth maps can be downloaded <a href="https://drive.google.com/file/d/1bh2WtAF6mhRhJIA1Gp68pmicakCqar4_/view?usp=share_link"> here </a>. You also need to download the camera instrinsic parameters found <a href="https://drive.google.com/file/d/1EINFlu6bJ32g8MuiH1PfWckB7nps36AO/view?usp=share_link"> camera.json </a> and the configuration files found <a href="https://drive.google.com/file/d/1rhyMusfFd6gX9dAv1l5IsVM0Xd06eMaL/view?usp=share_link">config.json</a>. The camera instrinisc parameters need to be placed inside "/content/Open3D/examples/python/reconstruction_system/optinonal/" and the config.json file needs to be inside "/content/Open3D/examples/python/reconstruction_system/datasets/016/".

The final step is to run open3D's 3D reconstruction system. 

In [ ]:
!python run_system.py --config ./datasets/016/config.json --make
%cd ./datasets/016/fragments/

The point cloud (output of the 3D reconstruction system) can be found inside /content/Open3D/examples/python/reconstruction_system/datasets/016/fragments/fragment_000.ply"
Use the following code below to visualize the final 3D reconstruction of the Chapel.

In [ ]:
import open3d as o3d
import numpy as np
from PIL import Image
import glob
import copy
from open3d_example import draw_geometries_flip

flip_transform = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]


def draw_geometries_flip(pcds):
    pcds_transform = []
    for pcd in pcds:
        pcd_temp = copy.deepcopy(pcd)
        pcd_temp.transform(flip_transform)
        pcds_transform.append(pcd_temp)
    o3d.visualization.draw_geometries(pcds_transform)


fragment_file = "fragment_000.ply"
pcd = o3d.io.read_point_cloud(fragment_file)
point_cloud_np = np.asarray(pcd.points)

# Save the point cloud to a .npy file
np.save("point_cloud.npy", point_cloud_np)

#draw_geometries_flip([pcd])
o3d.visualization.draw_geometries([pcd])
